In [5]:
import pika
import json
import time
from quads import QuadTree, Rect, Point
from collections import defaultdict

ImportError: cannot import name 'Rect' from 'quads' (C:\Users\jusch\anaconda3\lib\site-packages\quads.py)

In [4]:
!pip install quads

In [6]:
help(QuadTree)

Help on class QuadTree in module quads:

class QuadTree(builtins.object)
 |  QuadTree(center, width, height, capacity=None)
 |  
 |  Usage::
 |  
 |      >>> import quads
 |      >>> tree = quads.QuadTree(
 |      ...     (0, 0),  # The center point
 |      ...     10,  # The width
 |      ...     10,  # The height
 |      ... )
 |  
 |      >>> tree.insert((1, 2))
 |      True
 |  
 |      >>> tree.find((1, 2))
 |      Point(1, 2)
 |  
 |      >>> tree.find((4, -4))
 |      None
 |  
 |      >>> tree.nearest_neighbors((0, 1), count=2)
 |      [
 |          Point(1, 2),
 |          Point(4, -4),
 |      ]
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, point)
 |      Checks if a `Point` is found in the quadtree.
 |      
 |      > Note: This doesn't check if a point is within the bounds of the
 |      > tree, but if that *specific point* is in the tree.
 |      
 |      Args:
 |          point (Point|tuple|None): The point to check for.
 |      
 |      Returns:
 |          

In [ ]:
class WorldComponent:
    def __init__(self):
        # RabbitMQ und Tick
        self.connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        self.channel = self.connection.channel()
        self.channel.queue_declare(queue='world_update_queue')
        self.tick = 0

        # Datenstrukturen
        boundary = Rect(0, 0, 1000, 1000)  # Dimensionen der Welt
        self.terrain_quadtree = QuadTree(boundary, 4)  # Der zweite Parameter ist die Kapazität
        self.creatures_and_objects = defaultdict(list)
        self.layers = [self.terrain_quadtree, self.creatures_and_objects]

    def update(self):
        self.tick += 1

        # Update Layer Logik hier (z.B. neue Kreaturen oder Terrainänderungen)
        # ...

        # Beispiel: Füge einen neuen Punkt zum Quadtree hinzu
        point = Point(42, 42)  # Ersetze durch echte Daten
        self.terrain_quadtree.insert(point)

        new_world_state = {
            "tick": self.tick,
            "terrain": self.terrain_quadtree.export(),  # Nutze die Export-Methode von quads
            "entities": self.creatures_and_objects,
        }

        self.channel.basic_publish(exchange='',
                                   routing_key='world_update_queue',
                                   body=json.dumps(new_world_state))

    def run(self):
        while True:
            self.update()
            time.sleep(1)

# Instanz der WorldComponent-Klasse
world_component = WorldComponent()

# Start der dauerhaften Updateschleife
world_component.run()